In [ ]:
import boto3
import yaml
from pprint import pprint
import json
from dotenv import load_dotenv
import os
from pathlib import Path

In [ ]:
ENV_PATH = Path().absolute().parent.parent.joinpath('.env')

In [ ]:
assert load_dotenv(str(ENV_PATH))

# Delete Table

In [ ]:
TABLE_NAME = os.environ["TABLE"]

In [ ]:
def read_yaml_to_dict(path: str):
    with open(path, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

In [ ]:
db_schema = read_yaml_to_dict('./db-schema.yaml')

In [ ]:
# pprint(db_schema)

In [ ]:
def create_table(name: str):
    dynamodb = boto3.client('dynamodb')
    resp = dynamodb.create_table(
        TableName=name,
        AttributeDefinitions=db_schema['AttributeDefinitions'],
        KeySchema=db_schema['KeySchema'],
        GlobalSecondaryIndexes=db_schema['GlobalSecondaryIndexes'],
        ProvisionedThroughput=db_schema['ProvisionedThroughput'],
        Tags=db_schema['Tags']
    )
    pprint(resp)

In [ ]:
def list_all_tables():
    db = boto3.resource('dynamodb')
    tables = list(db.tables.all())
    print(tables)
    
list_all_tables()

# Create Table

In [ ]:
create_table(TABLE_NAME)

## Delete Example
Careful running that Function !
```python
def delete_table(name: str):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(name)
    table.delete()
    
delete_table('<>')
```

## Update Table Example

```python
# Add new Attribute
dynamodb.update_table(
    TableName=TABLE_NAME,
    AttributeDefinitions=[
        {
            "AttributeName": "parent_entity_pid",
            "AttributeType": "S"
        },
        {
            "AttributeName": "status_jid",
            "AttributeType": "S"
        }
    ],
    GlobalSecondaryIndexUpdates=[
        {"Create": db_schema["GlobalSecondaryIndexes"][1]}
    ]
)

# Delete Attribute
dynamodb.update_table(
    TableName=TABLE_NAME,
    GlobalSecondaryIndexUpdates=[{"Delete": {"IndexName": "JobStatusIndex"}}]
)
```